## Installs

In [ ]:
!pip install transformers peft bitsandbytes accelerate trl pydantic-settings scipy codebleu

In [2]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps trl peft accelerate bitsandbytes
!pip install -q xformers==0.0.22.post7

In [3]:
!pip install wandb -q

## Imports

In [1]:
from huggingface_hub import login
login(token="")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [5]:
import wandb
wandb.login(key="693b1be0a3e998a3f5d7f10070e76d51fd051f90", relogin=True)
wandb.init(name="idl-c++-finetune")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
wandb: Currently logged in as: sanssoft (hickups). Use `wandb login --relogin` to force relogin


In [2]:
# DEFINE QUANTIZATION HERE. Choose from ("none" | "8bit" | "4bit")
QUANTIZATION = "4bit"#DO NOT CHANGE
DATASET = "codeparrot/xlcost-text-to-code" #DO NOT CHANGE
TESTSIZE = 200

In [3]:
#CHANGE THE FOLLOWING
EXPERT = "cpp" #python, cpp, java, javascript, etc.
MODELTYPE = "FINETUNED" #BASE or FINETUNED or MOE
model_id = "mistralai/Mistral-7B-v0.1"
newmodel = f"mistral-7b-{EXPERT}-{MODELTYPE}-4bit"
loramodel = f"mistral-7b-{EXPERT}-LORA-4bit"
lorarepo = f'santoshsto/{loramodel}'
PARTITION = "C++-program-level" #java-program-level etc.

In [4]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [5]:
from datasets import load_dataset
data = load_dataset(DATASET, PARTITION)

In [6]:
next(iter(data["test"]))

{'text': 'Minimum sum possible by removing all occurrences of any array element | C ++ program for the above approach ; Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Driver code ; Input array ; Size of array',
 'code': '#include <bits/stdc++.h> NEW_LINE using namespace std ; int minSum ( int A [ ] , int N ) { map < int , int > mp ; int sum = 0 ; for ( int i = 0 ; i < N ; i ++ ) { sum += A [ i ] ; mp [ A [ i ] ] ++ ; } int minSum = INT_MAX ; for ( auto it : mp ) { minSum = min ( minSum , sum - ( it . first * it . second ) ) ; } return minSum ; } int main ( ) { int arr [ ] = { 4 , 5 , 6 , 6 } ; int N = sizeof ( arr ) / sizeof ( arr [ 0 ] ) ; cout << minSum ( arr , N ) << " STRNEWLINE " ; }'}

In [7]:
train_dataset = data["train"]
eval_dataset = data["validation"]
test_dataset = data["test"]

In [8]:
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'code'],
    num_rows: 9797
})
Dataset({
    features: ['text', 'code'],
    num_rows: 492
})
Dataset({
    features: ['text', 'code'],
    num_rows: 909
})


In [13]:
# Pre-define quantization configs

################## 4bit ##################
bb_config_4b = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
##########################################

################## 8bit ##################
bb_config_8b = BitsAndBytesConfig(
    load_in_8bit=True,
)
##########################################

def quantization_config(quantization):
    if quantization == "8bit":
        return bb_config_8b
    else:
        return bb_config_4b

In [14]:
%%capture
if QUANTIZATION == "none":
    model = AutoModelForCausalLM.from_pretrained(model_id).to("cuda")
else:
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config(QUANTIZATION))

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [18]:
base_prompt = """Write a method to output the prime factorization of 2023 in C++, C, and Java"""

In [19]:
base_tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_bos_token=True,
)

base_tokenizer.pad_token_id = base_tokenizer.eos_token_id

model_input = base_tokenizer(base_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(base_tokenizer.decode(model.generate(**model_input, max_new_tokens=256)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a method to output the prime factorization of 2023 in C++, C, and Java.

Prime factorization is a method of representing a number as a product of prime numbers.

For example, 2023 can be represented as 2023 = 3 * 3 * 17 * 17.

The prime factorization of 2023 is 3 * 3 * 17 * 17.

## Prime Factorization of 2023 in C++

The following is the C++ code to output the prime factorization of 2023.

```
#include <iostream>
using namespace std;

int main()
{
    int n = 2023;
    int i = 2;
    while (n > 1)
    {
        if (n % i == 0)
        {
            cout << i << " ";
            n = n / i;
        }
        else
        {
            i++;
        }
    }
    return 0;
}
```

## Prime Factorization of 2023 in C

The following is the C


## Training

In [20]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

def tokenize(prompt):
    tokenized = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [21]:
def process_prompt(data):
  new_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. {data['text']} [/INST] \\n {data['code']} </s>"""
  #new_prompt = f"""<s>[INST] {data["instruction"]} here are the inputs {data["input"]} [/INST] \\n {data["output"]} </s>"""
  return tokenize(new_prompt)

tokenized_train_ds = train_dataset.map(process_prompt)
tokenized_val_ds = eval_dataset.map(process_prompt)

Map: 100%|██████████| 492/492 [00:00<00:00, 1022.81 examples/s]


In [22]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_param_info(model):
    """
    Outputs trainable parameter information.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [23]:
config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_param_info(model)

trainable params: 170082304 || all params: 3922153472 || trainable%: 4.336452033664837


### Train

In [24]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [25]:
# Parallelization is possible if system is multi-GPU
if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True

tokenizer.pad_token = tokenizer.eos_token
steps_completed = 500
total_steps = 1156
remaining_steps = total_steps - steps_completed
# Training configs
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    args=transformers.TrainingArguments(
        output_dir = "./c++_results",
        warmup_steps=5,
        per_device_train_batch_size=16,
        gradient_checkpointing=True,
        gradient_accumulation_steps=2,
        num_train_epochs=4,
        max_steps=-1,
        warmup_ratio=0.03,
        learning_rate=2.5e-5,
        logging_steps=1,
        weight_decay=0.001,
        max_grad_norm=0.3,
        group_by_length = True,
        bf16=False,
        fp16=False,
        optim="paged_adamw_8bit",
        logging_dir="./logs",
        save_strategy="steps",
        save_steps=50,
        evaluation_strategy="steps",
        eval_steps=50,
        report_to="wandb",
        do_eval=True,
        # Set to True to resume training from the checkpoint
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# Silencing warnings. If using for inference, consider re-enabling.
model.config.use_cache = False

# Train!
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.887700,0.821006
100,0.780900,0.764365
150,0.767900,0.724816
200,0.710000,0.692274
250,0.690500,0.685657
300,0.742900,0.679823
350,0.680000,0.676013
400,0.657400,0.672963
450,0.715700,0.671500
500,0.672900,0.668590


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in

TrainOutput(global_step=1224, training_loss=0.683281694435411, metrics={'train_runtime': 64544.3816, 'train_samples_per_second': 0.607, 'train_steps_per_second': 0.019, 'total_flos': 8.75310654947328e+17, 'train_loss': 0.683281694435411, 'epoch': 3.99})

In [26]:
trainer.model.save_pretrained(newmodel)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [29]:
trainer.model.push_to_hub(loramodel)

adapter_model.safetensors: 100%|██████████| 1.20G/1.20G [00:24<00:00, 49.5MB/s]


CommitInfo(commit_url='https://huggingface.co/santoshsto/mistral-7b-cpp-LORA-4bit/commit/f650bdbc99a81bf99b7b86ba3f2197a8dc2d021c', commit_message='Upload model', commit_description='', oid='f650bdbc99a81bf99b7b86ba3f2197a8dc2d021c', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
tokenizer.push_to_hub(loramodel)

tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 4.00MB/s]


CommitInfo(commit_url='https://huggingface.co/santoshsto/mistral-7b-cpp-LORA-4bit/commit/172d4a56d4ad09ea784b965c43b9ceb339062444', commit_message='Upload tokenizer', commit_description='', oid='172d4a56d4ad09ea784b965c43b9ceb339062444', pr_url=None, pr_revision=None, pr_num=None)

In [31]:
wandb.finish()

eval/loss,█▆▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▂▁▂▃▁▃▂▂▃▂▃▂▂▂▂▁▂▂▃▃▅▆▇█
eval/samples_per_second,▇██▆█▆▇▇▆▇▆▇▇▇▇██▇▆▆▄▃▁▁
eval/steps_per_second,███▅████▅██████████▅▅▁▁▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▇▁▁▂▃▅▂▂▂▂▂▂▃▃▃▃▄▃▃▄▄▄▅▄▅▅▅▆▆▆▆▇▇▇▇▆███▇
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▅▄▄▃▃▂▂▃▂▃▃▂▃▂▃▂▂▂▂▂▂▁▂▂▂▁▂▂▁▁▃▃▁▁▂▂▂▂
train/total_flos,▁
train/train_loss,▁


## Merge LoRA to Base Model

In [9]:
# Pre-define quantization configs

################## 4bit ##################
bb_config_4b_eval = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
##########################################

################## 8bit ##################
bb_config_8b_eval = BitsAndBytesConfig(
    load_in_8bit=True,
)
##########################################

def quantization_config_eval(quantization):
    if quantization == "8bit":
        return bb_config_8b_eval
    else:
        return bb_config_4b_eval

In [10]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

if QUANTIZATION == "none":
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=False
    ).to("cuda")
else:
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config_eval(QUANTIZATION),
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=False
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [01:49<00:00, 54.84s/it]


In [11]:
torch.cuda.empty_cache()

In [13]:
model_to_merge = PeftModel.from_pretrained(base_model, '/home/ec2-user/idl_cpp/mistral-7b-c++-FINETUNED-4bit')
# merged_model = model_to_merge.merge_and_unload()
# merged_model.save_pretrained(newmodel)


In [ ]:
# merged_model.push_to_hub(newmodel, use_temp_dir=False)
# tokenizer.push_to_hub(newmodel, use_temp_dir=False)

## Finetuned Model Inferencing

In [16]:
eval_prompt = f"""<s>[INST] Write a c++ program to complete the following. For a given integer n, print out all its prime factors one on each line.
n = 30 [/INST] Code: </s>"""

input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model_to_merge.generate(input_ids=input_ids, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id, do_sample=True, top_p=0.9,temperature=0.5)
prediction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(f"Prompt:\n{eval_prompt}\n")
print(f"\nGenerated response:\n{prediction[len(eval_prompt):]}")
print('''\nGround truth:\ndef print_prime_factors(n):
  for i in range(2, n + 1):
    while n % i == 0:
      print(i)
      n //= i
print_prime_factors(n)''')

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Prompt:
<s>[INST] Write a c++ program to complete the following. For a given integer n, print out all its prime factors one on each line.
n = 30 [/INST] Code </s>


Generated response:
 \n #include <bits/stdc++.h> NEW_LINE using namespace std ; void printPrimeFactors ( int n ) { while ( n % 2 == 0 ) { cout << 2 << endl ; n = n / 2 ; } for ( int i = 3 ; i <= sqrt ( n ) ; i += 2 ) { while ( n % i == 0 ) { cout << i << endl ; n = n / i ; } } if ( n > 2 ) cout << n << endl ; } int main ( ) { int n = 30 ; printPrimeFactors ( n ) ; return 0 ; } -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- -------------------------------- --------------------------